<a href="https://colab.research.google.com/github/ignacioaranguren1/bd_2/blob/main/bd_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import pickle
import os
import keras_tuner

from tqdm import tqdm
from datetime import datetime
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from keras_tuner.tuners import RandomSearch

In [2]:
%matplotlib inline

datapath = '/Users/ignacioaranguren/bd_2/data'
os.chdir(datapath)

1.In the data used by Gu, Kelly and Xiu (RFS 2019 – provided in class), use a similar procedure to theirs to predict stock returns with neural networks. Start by finding a suitable baseline configuration, and use a validation procedure to pick optimal hyperparameters for three neural network models: One with 2 hidden layers, one with 3 hidden layers, and one with 4 hidden layers. 

In [3]:
data = pd.read_pickle('returns_chars_panel.pkl')
macro = pd.read_pickle('macro_timeseries.pkl')

In [4]:
def train_validation_test_split(data,train_end_date,validation_end_date):
  tmp = data.reset_index()
  train = tmp[tmp.date<=train_end_date].set_index(['date','permno'],drop=True)
  validation = tmp[(tmp.date>train_end_date) & (tmp.date<=validation_end_date)].set_index(['date','permno'],drop=True)
  test = tmp[tmp.date>validation_end_date].set_index(['date','permno'],drop=True)
  return train,validation,test

In [5]:
data_merged = pd.merge(data,macro,on=['date'])
datelist = list(set(data_merged['date']))
datelist.sort()
data_merged.set_index(['date','permno'],drop=True,inplace=True)

In [6]:
train_ratio = 0.5
validation_ratio = 0.25
train_date = datelist[int(len(datelist)*train_ratio)]
validation_date = datelist[int(len(datelist)*(train_ratio+validation_ratio))]
X = data_merged.iloc[:,3:].copy()
y = data_merged['excess_ret'].copy()

In [13]:
X_train,X_validation,X_test = train_validation_test_split(X,train_date,validation_date)
y_train,y_validation,y_test = train_validation_test_split(y,train_date,validation_date)

In [33]:
def keras_model(n_layers, units, dropout_rate, learning_rate):
    # Build model
    model = Sequential()
    model.add(layers.Input(shape=(105,)))
    for i in range(n_layers):
        model.add(layers.Dense(units=units[0][i], activation='relu'))
    model.add(layers.Dropout(rate=dropout_rate[0]))
    model.add(layers.Dense(1, activation='linear'))
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
        loss='mse')
    return model

class HyperRegressor(keras_tuner.HyperModel):
    def __init__(self, n_layers, *args, **kwargs):
        self.n_layers = n_layers
        super().__init__(*args, **kwargs)

    def build(self, hp):
        units=[hp.Int(f'units_{i + 1}',min_value=16,max_value=160,step=16) for i in range(self.n_layers)],
        dropout_rate=hp.Choice('dropout_rate', values=[0.2,0.4,0.6,0.8]),
        learning_rate = hp.Float("learning_rate", min_value=1e-4, max_value=1e-2, sampling="log")
        return keras_model(self.n_layers,
                           units, 
                           dropout_rate, 
                           learning_rate
                          )

    def fit(self, hp, model, x, y, validation_data, **kwargs):
        model.fit(x, y, **kwargs)
        x_val, y_val = validation_data
        y_pred = model.predict(x_val)
        # Return a single float to minimize.
        return np.mean((y_pred - y_val)**2)

In [37]:
MAX_TRIALS = 10
EXECUTION_PER_TRIAL = 3
EPOCHS = 10
BATCH_SIZE = 256

def tune_model(n_layers=2):
  tuner = RandomSearch(
        hypermodel=HyperRegressor(n_layers),
        max_trials=MAX_TRIALS,
        executions_per_trial=EXECUTION_PER_TRIAL,
        overwrite=True,
        directory='bd_2',
        project_name=f'NN_new_{n_layers}'
  )
  tuner.search(
      X_train.values, 
      y_train.values,
      validation_data=(X_validation.values, y_validation.values),
      batch_size=BATCH_SIZE,
      epochs=EPOCHS
  )
  return tuner

In [43]:
models=[]
parameters=[]
tuners=[]
for n in range(2,5):
    tuner = tune_model(n)
    parameters.append(tuner.get_best_hyperparameters)
    models.append(tuner.get_best_models(1)[0])
    tuners.append(tuner)

Trial 1 Complete [00h 00m 39s]
default_objective: 0.04027897386593732

Best default_objective So Far: 0.04027897386593732
Total elapsed time: 00h 00m 39s
INFO:tensorflow:Oracle triggered exit


In [44]:
models

In [45]:
results = {}
for i in range(3):
    train_result = models[i].evaluate(X_train.values,y_train.values,batch_size=256)
    test_result = models[i].evaluate(X_test.values,y_test.values,batch_size=256)
    results[f'model_{i}'] = {'train': train_result,'test': test_result}

4312/4312 [==============================] - 3s 768us/step - loss: 0.0256


In [46]:
results

{'model_0': {'train': 0.022297639399766922, 'test': 0.025631392374634743},
 'model_1': {'train': 0.022430259734392166, 'test': 0.02561713196337223},
 'model_2': {'train': 0.022570274770259857, 'test': 0.025640852749347687}}

In [ ]:
with open('parameters.pkl','wb') as f:
  pickle.dump(parameters,f,protocol=4)
with open('models.pkl','wb') as f:
  pickle.dump(models,f,protocol=4)
with open('tuners.pkl','wb') as f:
  pickle.dump(tuners,f,protocol=4)

2.Use test data to get an idea of the out of sample performance of each model. Convert the standard MSE metric for out of sample performance to the “R2 out of sample” metric that was discussed in class. Compare your results to those in Gu-Kelly-Xiu and comment on the differences. 

3.Pick the model that performs the best out of sample, and interpret its output by doing the following analysis of variable importance:
a.	First, for all stock characteristics, get variable importance by setting one predictor at a time to zero and finding the decrease in out of sample R2. Show a table of the 10 most important variables according to this measure, and give an economic interpretation. 


b.	Second, get a measure of the joint importance of all our “macro predictors” (i.e., those taken from Welch and Goyal 2008), by setting them all to zero and finding the decrease in out of sample R2. Comment on how important macroeconomic variables are relative to stock characteristics in predicting returns. 

c.	Repeat the two steps above, but by using a measure of the sensitivity of predictions to each input variable, as outlined in the lectures.

4.Fit a penalised linear model (LASSO) to the same data, using validation data to pick the best penalty (e.g., you can use the “sklearn” package in Python to do this easily). Compare its test data performance to the neural network. 

5.Suppose somebody tells you to collect 10 more micro or macro variables that can predict returns and are not in our current dataset. How would you choose those variables, based on the intuitions you have gained in this project?